According to the assignment spec, you can ignore tweets that:
- have 'null' or 'undefined' ('und') language
- have no location information
- are made outside of the Grid.

In [1]:
import json
import pandas as pd
import numpy as np

from IPython.display import display
from shapely.geometry import shape, MultiPolygon, Point, Polygon
import geopandas as gpd
import matplotlib

In order for geopandas library to work on windows machine, please download the following packages (which are suitable for your machine and python version e.g. GDAL-3.4.2-cp37-cp37m-win32.whl for 32bit python 3.7. My windows is 64bit but I have 32 bit python installed for some reason) in the order mentioned below from this website : https://www.lfd.uci.edu/~gohlke/pythonlibs/. On Linux or Mac, geopandas can be installed using conda or pip (we will have to verify that later on Spartan)

1. GDAL
2. pyproj
3. Fiona
4. Shapely
5. geopandas

# language code -> language name mapping

In [2]:
# https://datahub.io/core/language-codes

lang_map = pd.read_csv('./data/language-codes_csv.csv')
lang_map.loc[lang_map.shape[0]] = ['in', 'Indonesian']
lang_map.loc[lang_map.shape[0]] = ['iw', 'Former Hebrew']

lang_map = {code:name for code,name in zip(lang_map['alpha2'], lang_map['English'])}
lang_map

{'aa': 'Afar',
 'ab': 'Abkhazian',
 'ae': 'Avestan',
 'af': 'Afrikaans',
 'ak': 'Akan',
 'am': 'Amharic',
 'an': 'Aragonese',
 'ar': 'Arabic',
 'as': 'Assamese',
 'av': 'Avaric',
 'ay': 'Aymara',
 'az': 'Azerbaijani',
 'ba': 'Bashkir',
 'be': 'Belarusian',
 'bg': 'Bulgarian',
 'bh': 'Bihari languages',
 'bi': 'Bislama',
 'bm': 'Bambara',
 'bn': 'Bengali',
 'bo': 'Tibetan',
 'br': 'Breton',
 'bs': 'Bosnian',
 'ca': 'Catalan; Valencian',
 'ce': 'Chechen',
 'ch': 'Chamorro',
 'co': 'Corsican',
 'cr': 'Cree',
 'cs': 'Czech',
 'cu': 'Church Slavic; Old Slavonic; Church Slavonic; Old Bulgarian; Old Church Slavonic',
 'cv': 'Chuvash',
 'cy': 'Welsh',
 'da': 'Danish',
 'de': 'German',
 'dv': 'Divehi; Dhivehi; Maldivian',
 'dz': 'Dzongkha',
 'ee': 'Ewe',
 'el': 'Greek, Modern (1453-)',
 'en': 'English',
 'eo': 'Esperanto',
 'es': 'Spanish; Castilian',
 'et': 'Estonian',
 'eu': 'Basque',
 'fa': 'Persian',
 'ff': 'Fulah',
 'fi': 'Finnish',
 'fj': 'Fijian',
 'fo': 'Faroese',
 'fr': 'French',
 'fy'

# Open twitter json file

In [3]:
with open('./data/smallTwitter.json', 'r', encoding='utf-8') as f:
    small_twitter_data = json.load(f)

In [4]:
len(small_twitter_data['rows'])

4999

# extract key attributes

In [5]:
rows_dict = {'tweet_id': [], 
             'language': [],
#              'geo': [],
             'coordinates': []}

for i, tweet in enumerate(small_twitter_data['rows']):
    rows_dict['tweet_id'].append(tweet['id'])
    rows_dict['language'].append(tweet['doc']['metadata']['iso_language_code'])
#     rows_dict['geo'].append(tweet['doc']['geo'])
    rows_dict['coordinates'].append(tweet['doc']['coordinates'])
    
# coordiantes: (long, lat). this is what we want as it is in the same format as coordinates in sysGrid.json
# there is also 'geo' attribute in the json, which is in (lat,long) format

In [6]:
df_tweets = pd.DataFrame(rows_dict)

In [7]:
df_tweets.head()

,tweet_id,language,coordinates
0,1212161512334336000,es,None
1,1212161512770437121,en,None
2,1212161513361793024,en,None
3,1212161513580105733,en,None
4,1212161514112770048,en,None


# cleaning

In [8]:
# drop any rows with None/nan language or coordinates
df_tweets = df_tweets.dropna(subset=['language', 'coordinates'])

# drop undefined or null language rows
df_tweets = df_tweets[df_tweets['language']!='und']
df_tweets = df_tweets[df_tweets['language']!='null']

In [9]:
df_tweets

,tweet_id,language,coordinates
227,1212162687049883648,en,"{'type': 'Point', 'coordinates': [151.211, -33..."
891,1212166639598395394,en,"{'type': 'Point', 'coordinates': [151.20797, -..."
1082,1212167754888953856,en,"{'type': 'Point', 'coordinates': [151.27053, -..."
1095,1212167790028705793,en,"{'type': 'Point', 'coordinates': [151.17834389..."
1301,1212168948503863296,en,"{'type': 'Point', 'coordinates': [151.072247, ..."
1339,1212169394958323712,en,"{'type': 'Point', 'coordinates': [151.20797, -..."
2093,1212173497167794180,en,"{'type': 'Point', 'coordinates': [151.0655, -3..."
2358,1212175152949022720,en,"{'type': 'Point', 'coordinates': [151.2102003,..."
3073,1212179185520627712,en,"{'type': 'Point', 'coordinates': [151.20797, -..."
3317,1212180379873898497,en,"{'type': 'Point', 'coordinates': [151.20797, -..."


In [10]:
# extract (long,lat) coordinates 
df_tweets['coordinates'] = df_tweets['coordinates'].apply(lambda x: x['coordinates'])
# map language code to language name. if language code is not in our mapping dict, replace it with None 
df_tweets['language'] = df_tweets['language'].apply(lambda x: lang_map[x] if x in lang_map else None)

# remove rows where language or coordinates is None.
df_tweets = df_tweets.dropna(subset=['language', 'coordinates'])

id, language, coordinates should be all key info we need... (don't even need tweet id maybe?)

In [11]:
df_tweets

,tweet_id,language,coordinates
227,1212162687049883648,English,"[151.211, -33.86]"
891,1212166639598395394,English,"[151.20797, -33.86751]"
1082,1212167754888953856,English,"[151.27053, -33.9005]"
1095,1212167790028705793,English,"[151.17834389, -33.93467816]"
1301,1212168948503863296,English,"[151.072247, -33.8473998]"
1339,1212169394958323712,English,"[151.20797, -33.86751]"
2093,1212173497167794180,English,"[151.0655, -33.81967]"
2358,1212175152949022720,English,"[151.2102003, -33.85999135]"
3073,1212179185520627712,English,"[151.20797, -33.86751]"
3317,1212180379873898497,English,"[151.20797, -33.86751]"


# CODE THAT DETERMINES THE CELL ID FOR EACH TWEET

## Hannan can fill this part in

My rough idea:

- for each row (these tweets all have both valid language and location info already) in `df_tweets`,
    - process the coordinates (e.g. [151.03, -33.88]) and determine which cell/grid-location id it should belong to, using the `sysGrid.json` file.
    
So after this section of code chunk, we will have another column named `cell` with values A1, A2, ..., D4.

In [13]:
with open('./data/sydGrid-2.json', 'r', encoding='utf-8') as f:
    syd_grid= json.load(f)

syd_grid_coorindates = []
syd_grid_id = []
for features in syd_grid['features']:
    poly=Polygon(features['geometry']['coordinates'][0])
    syd_grid_coorindates+=[poly]
    syd_grid_id.append(features['properties']['id'])
geodata=gpd.GeoDataFrame()
geodata['cells_id']= syd_grid_id
geodata['geometry'] = syd_grid_coorindates
coords=[Point(xy) for xy in df_tweets['coordinates']]
gdf_locations = gpd.GeoDataFrame(df_tweets,geometry=coords)
# that requires rtree or pygeos package and can be installed using pip. rtree is not working for some reason, pygeos
# work but gives out compatibility issues warnings with shapely packge. On windows, shapely was installed
# indepenedenly to install geopandas. In linux env geopandas and all its dependencies will be installed using either
# conda and pip and therefore this compatibility issue will be resolved. for more information, visit
# https://github.com/geopandas/geopandas/issues/2355
tweets_with_cells=gpd.sjoin(gdf_locations,geodata,how='left',predicate='within')
tweets_with_cells = tweets_with_cells.dropna(subset=['cells_id'])

Cells don't have values A1, A2,....D4. They have numerical ids as described in the sydGrid.json file

In [14]:
tweets_with_cells

,tweet_id,language,coordinates,geometry,index_right,cells_id
227,1212162687049883648,English,"[151.211, -33.86]",POINT (151.21100 -33.86000),4.0,19.0
891,1212166639598395394,English,"[151.20797, -33.86751]",POINT (151.20797 -33.86751),4.0,19.0
1082,1212167754888953856,English,"[151.27053, -33.9005]",POINT (151.27053 -33.90050),0.0,23.0
1095,1212167790028705793,English,"[151.17834389, -33.93467816]",POINT (151.17834 -33.93468),4.0,19.0
1301,1212168948503863296,English,"[151.072247, -33.8473998]",POINT (151.07225 -33.84740),5.0,18.0
1339,1212169394958323712,English,"[151.20797, -33.86751]",POINT (151.20797 -33.86751),4.0,19.0
2358,1212175152949022720,English,"[151.2102003, -33.85999135]",POINT (151.21020 -33.85999),4.0,19.0
3073,1212179185520627712,English,"[151.20797, -33.86751]",POINT (151.20797 -33.86751),4.0,19.0
3317,1212180379873898497,English,"[151.20797, -33.86751]",POINT (151.20797 -33.86751),4.0,19.0
3885,1212183787771170817,English,"[151.0355168, -33.8392224]",POINT (151.03552 -33.83922),9.0,14.0


# Count # of languages and # of tweets for each language, in each cell

Previous section should have added an extra column with cell id for each tweet.

I'll create a mock column for now and also some additional rows to add simulate other languages.

In [17]:
# count # of languages in each cell
df_results = pd.DataFrame(tweets_with_cells.groupby(['cells_id']).size()).reset_index().rename(columns={0:'#Total Tweets'})
df_results

,cells_id,#Total Tweets
0,14.0,1
1,18.0,2
2,19.0,10
3,23.0,1


In [19]:
# count # of occurences for each language in each cell
pd.DataFrame(tweets_with_cells.groupby(['cells_id', 'language'], as_index=False).size())

,cells_id,language,size
0,14.0,English,1
1,18.0,English,2
2,19.0,English,10
3,23.0,English,1


# thoughts
- This notebook is quick prototype only, will need to move code into a script.
    - Feel free to criticize/change/suggest imporvements to any part of my code/thoughts.
    - Need to incorporate MPI style code after.
- Can probably use MPI to split the twitter dataset into evenly-sized chunks (e.g. if twitter data size is 1000 and we have 10 processors, each processor can process 100 rows/tweets)?
    - Need to research if 1-node, 8-cores and 2-nodes, 4-cores each need to be handled differently.
- Each processor can follow an identical procedure (shown in this notebook), and then at the end, we could aggregate all the final dataframe language counts into the format shown in the assignment spec?
    - Will just need to assign different section of the twitter dataset to each processor, and then aggregate at the end?

In [ ]:
df_cell_lang_size = df_tweets.groupby(['cell', 'language'], as_index = False).size().astype('str')
df_cell_lang_size['lang-size'] = df_cell_lang_size['language'] + '-' + df_cell_lang_size['size']
df_cell_lang_size

In [ ]:
pd.DataFrame(df_cell_lang_size.groupby(['cell'])['lang-size'].apply(','.join))